In [1]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier, export_text, plot_tree, sns
import constants
import utils

In [2]:
data

,Systemic Illness,Rectal Pain,Sore Throat,Penile Oedema,Oral Lesions,Solitary Lesion,Swollen Tonsils,HIV Infection,Sexually Transmitted Infection,MonkeyPox
Patient_ID,,,,,,,,,,
P0,None,False,True,True,True,False,True,False,False,Negative
P1,Fever,True,False,True,True,False,False,True,False,Positive
P2,Fever,False,True,True,False,False,False,True,False,Positive
P3,None,True,False,False,False,True,True,True,False,Positive
P4,Swollen Lymph Nodes,True,True,True,False,False,True,True,False,Positive
...,...,...,...,...,...,...,...,...,...,...
P24995,None,True,True,False,True,True,False,False,True,Positive
P24996,Fever,False,True,True,False,True,True,True,True,Positive
P24997,None,True,True,False,False,True,True,False,False,Positive


In [17]:
series = data[constants.MONKEY_POX].value_counts()
#series2,
# TODO: Try to use countplot and change colors to normal blue and orange
plt.bar(series.index, series.values, color=['blue', 'orange'])
# sns.countplot(data[constants.MONKEY_POX])
plt.show()

In [4]:
df_dict = dict.fromkeys(data[constants.SYSTEMIC_ILLNESS].unique())
# find better solution
for index in data.index:
    if data['MonkeyPox'][index] == 'Positive':
        df_dict[data[constants.SYSTEMIC_ILLNESS][index]] = int(df_dict[data[constants.SYSTEMIC_ILLNESS][index]] or 0) + 1
df_dict

{'None': 3324,
 'Fever': 4780,
 'Swollen Lymph Nodes': 4581,
 'Muscle Aches and Pain': 3224}

In [5]:
df = pd.DataFrame(df_dict.items())
df.rename(columns={0: 'illness', 1: 'quantity'}, inplace=True)
plt.bar(df['illness'], df['quantity'])
plt.title('Cases of monkeypox seperated by {}'.format(constants.SYSTEMIC_ILLNESS))
plt.xlabel('Illness')
plt.ylabel('Occurrence')
plt.show()

In [13]:
# TODO: Change legend and axises
test_df = data.groupby(by=[constants.SYSTEMIC_ILLNESS,constants.MONKEY_POX])[constants.SYSTEMIC_ILLNESS].count()
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX])
plt.show()

In [ ]:
sns.pairplot(data.head(100), hue='MonkeyPox')

In [ ]:
simple_dict = {'None': 1, 'Fever': 2, 'Swollen Lymph Nodes': 3, 'Muscle Aches and Pain': 4}
data[constants.SYSTEMIC_ILLNESS] = [simple_dict[item] for item in data[constants.SYSTEMIC_ILLNESS]]
data_tree = data.iloc[:,:-1]
target = data.iloc[:,-1:]
ran_stream = 23
x_train, x_test, y_train, y_test = train_test_split(data_tree, target, random_state=ran_stream)

model = GaussianNB()
model.fit(x_train, y_train)
y_prediction = model.predict(x_test)
print(100 * accuracy_score(y_test, y_prediction))

In [ ]:
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=5)
model_tree.fit(data_tree, target)
text_representation = export_text(model_tree, feature_names=data_tree.columns.values.tolist())
figure = plt.figure(figsize=(10,8))
plot_tree(model_tree, feature_names=data_tree.columns, filled=True, rounded=True)
plt.show()


In [ ]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.make_df_true(symptoms, symptoms_df)

pos_lst, negative_lst = utils.create_outcome_list(symptoms, symptoms_df)
df1 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': pos_lst})
df2 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': negative_lst})
df1['outcome'] = 'Positive'
df2['outcome'] = 'Negative'
res = pd.concat([df1, df2])
sns.barplot(x='Symptoms', y='Number of infections', data=res, hue='outcome')
plt.show()